# Classificador de Sentiments a Xarxes Socials en Català (CSXSC): Dataset

**Author:** Daniel Arias Cámara  
**Date:** July 2025  

**Description:**  This notebook aims to build a high-quality dataset for fine-tuning the **CSXSC** model. The dataset is constructed by combining trusted data sources, including structured sentiment corpora and translated social media content. Details on data origin and preprocessing steps are provided in the sections below.


## 1. GuiaCat Dataset

**Description:** This dataset consists of 5,750 restaurant reviews in Catalan, sourced from the GuiaCat platform. Each review includes individual ratings for service, food, price-quality ratio, and atmosphere, along with an overall average score.

**Access:** [projecte-aina/GuiaCat on Hugging Face](https://huggingface.co/datasets/projecte-aina/GuiaCat)

**Source:** Aina Project

**Notes:**  
The dataset is divided into three subsets:  
- **Train:** 4,750 rows  
- **Validation:** 500 rows  
- **Test:** 500 rows  

The original fields are: Service, Food, Price-quality, Environment, Avg, Text, and Label.  
For our purposes, we retain only the Text and Label fields, discarding the rest.

The Label field includes five sentiment categories:  
- Molt bo (Very good)  
- Bo (Good)  
- Regular (Average)  
- Dolent (Bad)  
- Molt dolent (Very bad)

These are grouped into three classes for sentiment classification:  
- **Positive:** Molt bo and Bo  
- **Neutral:** Regular  
- **Negative:** Dolent and Molt dolent


In [ ]:
import subprocess
from typing import Dict, List

try:
    import pandas as pd
    from datasets import load_dataset, concatenate_datasets
except ImportError:
    subprocess.check_call(["pip", "install", "-q", "pandas", "datasets", "pyarrow"])
    import pandas as pd
    from datasets import load_dataset, concatenate_datasets

COLUMNS_TO_KEEP: List[str] = ["text", "label"]
DATASET_NAME: str = "projecte-aina/GuiaCat"
CSV_FILENAME: str = "guiacat.csv"

def relabel_opinion(opinion: Dict) -> Dict:
    label = opinion["label"].lower()
    if label in ["molt bo", "bo"]:
        opinion["label"] = "positive"
    elif label == "regular":
        opinion["label"] = "neutral"
    elif label in ["dolent", "molt dolent"]:
        opinion["label"] = "negative"
    return opinion

def process_and_combine_dataset(dataset_name: str) -> pd.DataFrame:
    print(f"Loading and processing '{dataset_name}'")
    raw_dataset = load_dataset(dataset_name)
    
    processed_splits = []
    for split in raw_dataset:
        processed_split = raw_dataset[split].map(relabel_opinion)
        drop_columns = [col for col in processed_split.column_names if col not in COLUMNS_TO_KEEP]
        processed_splits.append(processed_split.remove_columns(drop_columns))
    

    combined_dataset = concatenate_datasets(processed_splits)
    return combined_dataset.to_pandas()


guiacat_df = process_and_combine_dataset(DATASET_NAME)
guiacat_df.to_csv(CSV_FILENAME, index=False)

total_rows = len(guiacat_df)
label_dist = guiacat_df['label'].value_counts(normalize=True) * 100

positive_pct = label_dist.get('positive', 0)
negative_pct = label_dist.get('negative', 0)
neutral_pct = label_dist.get('neutral', 0)

print("\nFinal Dataset Distribution")
print(
    f"Total Rows: {total_rows}\n"
    f"Distribution: {positive_pct:.1f}% Positive, "
    f"{negative_pct:.1f}% Negative, "
    f"{neutral_pct:.1f}% Neutral"
)

/home/user/anaconda3/envs/ai/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading and processing 'projecte-aina/GuiaCat'

Final Dataset Distribution
Total Rows: 5750
Distribution: 94.3% Positive, 3.6% Negative, 2.1% Neutral


## 2. Catalan Structured Sentiment Analysis (CaSSA) Dataset

**Description:** The CaSSA dataset contains 6,400 reviews and forum messages in Catalan, annotated at the fine-grained level with polar expressions. Each text instance is labeled with all the sentiment expressions it contains. For each polar expression, the annotation includes the **expression itself**, the **target** (i.e., the object of the sentiment), and the **source** (i.e., the subject expressing the sentiment). In total, 25,453 polar expressions have been annotated.

**Access:** [projecte-aina/CaSSA on Hugging Face](https://huggingface.co/datasets/projecte-aina/CaSSA-catalan-structured-sentiment-analysis)

**Source:** Aina Project

**Notes:**

Each instance in the dataset is a text. For each text, there can be 0 to unlimited polar expressions, which are contained in the "opinions" field. Each opinion contains a source, a target, a polar expression, a polarity value and an intensity value.

To convert this structured information into a single sentiment label per text, we apply the following strategy:
- Count all Positive, Negative, and Neutral polarities per opinion.
- Assign the sentiment label based on the dominant polarity.
  - If Positive polar expressions are the majority: **Positive**.
  - If Negative polar expressions dominate: **Negative**.
  - In case of a tie or no polar expressions: **Neutral**.

In [2]:
import subprocess
from typing import Dict, List

try:
    import pandas as pd
    from datasets import load_dataset
except ImportError:
    subprocess.check_call(["pip", "install", "-q", "pandas", "datasets", "pyarrow"])
    import pandas as pd
    from datasets import load_dataset

DATASET_NAME: str = "projecte-aina/CaSSA-catalan-structured-sentiment-analysis"
CSV_FILENAME: str = "cassa.csv"
COLUMNS_TO_KEEP: List[str] = ["text", "label"]

def relabel_from_opinions(item: Dict) -> Dict:
    pos = neg = neu = 0
    for opinion in item.get("opinions", []):
        polarity = (opinion.get("Polarity") or "").strip().lower()
        if polarity == "positive":
            pos += 1
        elif polarity == "negative":
            neg += 1
        elif polarity == "neutral":
            neu += 1

    if pos > neg and pos > neu:
        label = "positive"
    elif neg > pos and neg > neu:
        label = "negative"
    else:
        label = "neutral"

    return {"text": item["text"], "label": label}

def process_cassa_dataset(dataset_name: str) -> pd.DataFrame:
    print(f"Loading and processing '{dataset_name}'...")

    raw_dataset = load_dataset(dataset_name)["train"]
    processed_dataset = raw_dataset.map(relabel_from_opinions)
    
    drop_columns = [col for col in processed_dataset.column_names if col not in COLUMNS_TO_KEEP]
    cleaned_dataset = processed_dataset.remove_columns(drop_columns)
    
    return cleaned_dataset.to_pandas()

cassa_df = process_cassa_dataset(DATASET_NAME)
cassa_df.to_csv(CSV_FILENAME, index=False)

total_rows = len(cassa_df)
label_dist = cassa_df['label'].value_counts(normalize=True) * 100

positive_pct = label_dist.get('positive', 0)
negative_pct = label_dist.get('negative', 0)
neutral_pct = label_dist.get('neutral', 0)

print("\nFinal Dataset Distribution")
print(
    f"Total Rows: {total_rows}\n"
    f"Distribution: {positive_pct:.1f}% Positive, "
    f"{negative_pct:.1f}% Negative, "
    f"{neutral_pct:.1f}% Neutral"
)

Loading and processing 'projecte-aina/CaSSA-catalan-structured-sentiment-analysis'...

Final Dataset Distribution
Total Rows: 6400
Distribution: 64.0% Positive, 9.5% Negative, 26.5% Neutral


## 3. GoEmotions Dataset

## GoEmotions Dataset Processing

**Description:**  The **GoEmotions** dataset is a large-scale human-annotated corpus of 58k English Reddit comments labeled for 27 emotion categories plus neutral. It was developed by Google AI to support fine-grained sentiment and emotion classification in user-generated content.  
Each comment may have one or multiple labels, making it suitable for multilabel classification tasks.

**Access:** [Kaggle - GoEmotions Dataset](https://www.kaggle.com/datasets/debarshichanda/goemotions)  
**Source:** Google AI  

### Notes

The GoEmotions dataset includes annotations for 27 fine-grained emotion categories plus a neutral class.  
Since each Reddit comment can have multiple emotions, we adopt a two-step mapping strategy to simplify it into three sentiment categories: Positive, Negative, and Neutral.

#### **Step 1: Mapping Emotions to Broad Sentiment Groups**

- **Positive**: amusement, excitement, joy, love, desire, optimism, caring, pride, admiration, gratitude, relief, approval.  
- **Negative**: fear, nervousness, remorse, embarrassment, disappointment, sadness, grief, disgust, anger, annoyance, disapproval.  
- **Ambiguous**: realization, surprise, curiosity, confusion.

#### **Step 2: Decision Rule for Classification**

For each comment, count how many mapped emotions belong to each group and apply:

1. **Positive majority**: classify as **Positive**.  
2. **Negative majority**: classify as **Negative**.  
3. **Tie or no mapped emotions**: classify as **Neutral**.

### Translation to Catalan

After sentiment classification, all reviews are translated into Catalan using the  
[**Aina Project English–Catalan Translator**](https://huggingface.co/projecte-aina/aina-translator-ca-en),  
a machine translation model trained specifically for English to Catalan.

### Translation Quality Check

To ensure translations are high-quality:

1. Use the [**Salamandra 7B Instruct**](https://huggingface.co/BSC-LT/salamandra-7b-instruct) model.  
   - **Task 1:** Read the translated review and rate the translation quality on a **1–5 scale**,  
     considering the informal nature of social media (emojis, slang, grammar errors).  
   - **Task 2:** Justify the score, explaining why it was rated that way.

2. **Filtering:**  
   - Reviews scoring **below 3** are **discarded**.  
   - Justifications are logged for transparency.

In [3]:
import subprocess
import pandas as pd
import os
import gc
import torch

POSITIVE_SAMPLES_TO_ADD = 100
TARGET_DISTRIBUTION = {"positive": 0.40, "negative": 0.30, "neutral": 0.30}
SALAMANDRA_PATH = "/home/user/Escritorio/TFM/salamandra-7b-instruct"
NUM_SAMPLES_TO_EVALUATE = 500

for package in ["datasets", "pandas", "tqdm", "ctranslate2", "sentencepiece", "huggingface_hub", "transformers", "torch"]:
    try:
        __import__(package.split("[")[0])
    except ImportError:
        print(f"Installing required library: {package}")
        subprocess.check_call(["pip", "install", "-q", package])

from datasets import load_dataset
from tqdm.auto import tqdm
import ctranslate2
import sentencepiece as spm
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer, AutoModelForCausalLM

emotion_id2label = ["admiration","amusement","anger","annoyance","approval","caring","confusion","curiosity","desire","disappointment","disapproval","disgust","embarrassment","excitement","fear","gratitude","grief","joy","love","nervousness","optimism","pride","realization","relief","remorse","sadness","surprise","neutral"]
sentiment_map = {"positive": {"amusement","excitement","joy","love","desire","optimism","caring","pride","admiration","gratitude","relief","approval"},"negative": {"fear","nervousness","remorse","embarrassment","disappointment","sadness","grief","disgust","anger","annoyance","disapproval"},"ambiguous": {"realization","surprise","curiosity","confusion"}}

def parse_labels(labels_str):
    if isinstance(labels_str, list): return labels_str
    try: return [int(i) for i in str(labels_str).split(',')]
    except (ValueError, TypeError): return []

def classify_sentiment(emotion_ids):
    counts = {"positive": 0, "negative": 0, "ambiguous": 0}
    for eid in emotion_ids:
        if isinstance(eid, int) and eid < len(emotion_id2label):
            emotion = emotion_id2label[eid]
            for category, emotions in sentiment_map.items():
                if emotion in emotions: counts[category] += 1; break
    if counts["positive"] > counts["negative"] and counts["positive"] > counts["ambiguous"]: return "positive"
    elif counts["negative"] > counts["positive"] and counts["negative"] > counts["ambiguous"]: return "negative"
    else: return "neutral"

def add_sentiment_label(example):
    parsed_ids = parse_labels(example["labels"])
    example["sentiment_label"] = classify_sentiment(parsed_ids)
    return example

def initialize_models():
    print("Initializing all models.")
    model_dir = snapshot_download(repo_id="projecte-aina/aina-translator-en-ca", revision="main")
    sp_model_path = os.path.join(model_dir, "spm.model")
    sp = spm.SentencePieceProcessor()
    sp.load(sp_model_path)
    translator = ctranslate2.Translator(model_dir, device="auto")

    print("  - AINA translator initialized.")
    gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    eval_tokenizer = AutoTokenizer.from_pretrained(SALAMANDRA_PATH)
    eval_model = AutoModelForCausalLM.from_pretrained(SALAMANDRA_PATH, device_map="auto", torch_dtype=torch.float16)
    print("  - Salamandra evaluator LLM initialized.")
    return sp, translator, eval_tokenizer, eval_model

def translate_en_to_ca(text, sp, translator):
    try:
        tokens = sp.encode(text, out_type=str)
        translation = translator.translate_batch([tokens])
        return sp.decode(translation[0][0]["tokens"])
    except Exception: return ""

def evaluate_translation(text_ca, tokenizer, model):
    score_prompt = (
        "Ets un avaluador de qualitat de traducció en català. "
        "Et donaré un text en català procedent de xarxes socials, "
        "traduït automàticament des de l'anglès. "
        "Avalua la qualitat de la traducció en una escala del 1 (molt dolenta) al 5 (excel·lent). "
        "Ignora errors ortogràfics menors, abreviatures o estil informal. "
        "Tingues en compte que el text pot ser informal o col·loquial ja que prové d'opinions de xarxes socials. "
        "Respon només amb el número."
        f"\n\nText: {text_ca}"
    )
    messages_score = [{"role": "user", "content": score_prompt}]
    chat_text_score = tokenizer.apply_chat_template(messages_score, add_generation_prompt=True, tokenize=False)
    inputs_score = tokenizer(chat_text_score, return_tensors="pt").to(model.device)
    outputs_score = model.generate(**inputs_score, max_new_tokens=5, temperature=0.1)
    score_text = tokenizer.decode(outputs_score[0][len(inputs_score["input_ids"][0]):], skip_special_tokens=True).strip()
    try: score = int(score_text[0])
    except (ValueError, IndexError): score = 3
    explanation = ""
    if score <= 2:
        explanation_prompt = (
            "El text següent és una traducció automàtica del anglès al català. "
            "La seva qualitat s'ha valorat amb una puntuació baixa (≤ 2) "
            "en una escala de 1 a 5. Explica breument per què podria ser de baixa qualitat, "
            "centrant-te en problemes de traducció i no en el contingut."
            f"\n\nText: {text_ca}"
        )
        messages_explanation = [{"role": "user", "content": explanation_prompt}]
        chat_text_expl = tokenizer.apply_chat_template(messages_explanation, add_generation_prompt=True, tokenize=False)
        inputs_expl = tokenizer(chat_text_expl, return_tensors="pt").to(model.device)
        outputs_expl = model.generate(**inputs_expl, max_new_tokens=100, temperature=0.3)
        explanation = tokenizer.decode(outputs_expl[0][len(inputs_expl["input_ids"][0]):], skip_special_tokens=True).strip()
    return score, explanation

def classify_and_analyze_goemotions():
    print("- Step 1: Analyzing GoEmotions Dataset")
    goemotions_train = load_dataset("go_emotions", "simplified")["train"]
    classified_ds = goemotions_train.map(add_sentiment_label, batched=False)
    return classified_ds

def analyze_base_datasets():
    print("\n- Step 2: Analyzing Base Datasets (cassa.csv + guiacat.csv)")
    try:
        combined_df = pd.concat([pd.read_csv("cassa.csv"), pd.read_csv("guiacat.csv")], ignore_index=True)
        label_counts = combined_df['label'].value_counts().to_dict()
        print(f"Combined base dataset has {len(combined_df):,} rows. Distribution: {label_counts}")
        return label_counts
    except FileNotFoundError as e:
        print(f"Error: {e}. Ensure 'cassa.csv' and 'guiacat.csv' are in the root path.")
        return None

def collect_and_translate_goemotions(classified_ds, needed_counts, sp, translator):
    print("\n- Step 3: Collecting and Translating from GoEmotions")
    collected_samples = []
    goemotions_iterator = iter(classified_ds)
    with tqdm(total=sum(needed_counts.values()), desc="Collecting samples") as pbar:
        while sum(needed_counts.values()) > 0:
            try: row = next(goemotions_iterator)
            except StopIteration:
                print("\nWarning: Reached end of GoEmotions dataset before collecting all samples.")
                break
            sentiment = row["sentiment_label"]
            if needed_counts.get(sentiment, 0) > 0:
                translated_text = translate_en_to_ca(row["text"], sp, translator)
                if translated_text:
                    collected_samples.append({"text": translated_text, "label": sentiment})
                    needed_counts[sentiment] -= 1
                    pbar.update(1)
    return pd.DataFrame(collected_samples)

def filter_bad_translations(df, num_samples, tokenizer, model):
    print(f"\n- Step 4: Evaluating {num_samples} random translations before saving")
    if len(df) == 0: return df
    sample_df = df.sample(n=min(num_samples, len(df)), random_state=42)
    bad_texts = set()
    for _, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc="Evaluating translations"):
        score, explanation = evaluate_translation(row["text"], tokenizer, model)
        if score <= 2:
            print("\nDiscarded Translation:")
            print(f"Text: {row['text']}")
            print(f"Reason: {explanation}\n")
            bad_texts.add(row["text"])
    if bad_texts:
        print(f"Removing {len(bad_texts)} bad translations from dataset")
        df = df[~df["text"].isin(bad_texts)]
    return df

if __name__ == "__main__":
    classified_goemotions = classify_and_analyze_goemotions()
    base_counts = analyze_base_datasets()
    if base_counts:
        current_pos, current_neg, current_neu = base_counts.get("positive",0), base_counts.get("negative",0), base_counts.get("neutral",0)
        final_pos_count = current_pos + POSITIVE_SAMPLES_TO_ADD
        total_final_size = final_pos_count / TARGET_DISTRIBUTION["positive"]
        needed_counts = {
            "positive": POSITIVE_SAMPLES_TO_ADD,
            "negative": max(0, int(total_final_size * TARGET_DISTRIBUTION["negative"] - current_neg)),
            "neutral": max(0, int(total_final_size * TARGET_DISTRIBUTION["neutral"] - current_neu))
        }
        print("\nCollection Plan:")
        for label, count in needed_counts.items(): print(f"  Collect {label.capitalize()}: {count:,} rows")
        
        sp_translator, ctranslate_translator, eval_tokenizer, eval_model = initialize_models()
        
        goemotions_df = collect_and_translate_goemotions(
            classified_goemotions, 
            needed_counts.copy(), 
            sp_translator, 
            ctranslate_translator
        )
        
        goemotions_df = filter_bad_translations(goemotions_df, NUM_SAMPLES_TO_EVALUATE, eval_tokenizer, eval_model)
        
        output_filename = "goemotions.csv"
        goemotions_df.to_csv(output_filename, index=False)
        print(f"\nSaved {len(goemotions_df):,} cleaned samples to '{output_filename}'")
        
    print("\nFull pipeline finished.")


- Step 1: Analyzing GoEmotions Dataset

- Step 2: Analyzing Base Datasets (cassa.csv + guiacat.csv)
Combined base dataset has 12,150 rows. Distribution: {'positive': 9517, 'neutral': 1815, 'negative': 818}

Collection Plan:
  Collect Positive: 100 rows
  Collect Negative: 6,394 rows
  Collect Neutral: 5,397 rows
Initializing all models.


Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 102717.65it/s]


  - AINA translator initialized.


/home/user/anaconda3/envs/ai/lib/python3.13/site-packages/accelerate/utils/modeling.py:1582: UserWarning: Current model requires 128 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.20s/it]


  - Salamandra evaluator LLM initialized.

- Step 3: Collecting and Translating from GoEmotions


...


Discarded Translation:
Text: És lletja, així que això podria afegir alguns mesos més.
Reason: La frase "És lletja" probablement es refereix a alguna cosa física o visualment desagradable. Aquesta expressió pot tenir connotacions negatives i afectar la percepció d’una persona sobre un objecte o situació concret. En aquest cas, l'ús d'"afegir uns quants mesos més" sembla un intent de suavitzar les conseqüències negatives associades a la descripció inicial ("lletja"). No obstant això, aquesta expressió també té una certa quantitat de sarcasme



Evaluating translations:  99%|█████████▉| 497/500 [10:47:12<03:41, 73.73s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Evaluating translations: 100%|█████████▉| 498/500 [10:49:51<03:18, 99.44s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Discarded Translation:
Text: Necessitàvem alguna manera de torturar un tipus per poder crear una religió.
Reason: La traducció pot tenir errors gramaticals o sintàctics que afecten la comprensió del missatge original. També pot haver-hi faltes d'ortografia o paraules mal traduïdes que afectin l'exactitud de la traducció. En general, sembla que hi ha hagut poca atenció als detalls durant el procés de traducció.



Evaluating translations: 100%|██████████| 500/500 [10:52:08<00:00, 78.26s/it]

Removing 50 bad translations from dataset

Saved 11,841 cleaned samples to 'goemotions.csv'

Full pipeline finished.


## 4. Create the Final Dataset

The final step consists of generating the training, validation, and test splits. After combining the previously processed datasets, the rows are shuffled to ensure randomness and reduce potential ordering biases. The data is then divided into three subsets: 80% for training, 10% for validation, and 10% for testing. Finally, the resulting splits are stored as separate CSV files named `train.csv`, `validation.csv`, and `test.csv`.

In [ ]:
import pandas as pd
import subprocess

try:
    from sklearn.model_selection import train_test_split
except ImportError:
    subprocess.check_call(["pip", "install", "-q", "scikit-learn"])
    from sklearn.model_selection import train_test_split

INPUT_FILES = ["cassa.csv", "goemotions.csv", "guiacat.csv"]
OUTPUT_FILES = {
    "train": "train.csv",
    "validation": "validation.csv",
    "test": "test.csv"
}
RANDOM_STATE = 42

def print_distribution(df, name):
    total_rows = len(df)
    label_dist = df['label'].value_counts(normalize=True) * 100
    
    pos_pct = label_dist.get('positive', 0)
    neg_pct = label_dist.get('negative', 0)
    neu_pct = label_dist.get('neutral', 0)
    
    print(f"'{name}' ({total_rows:,} rows):")
    print(f"  Distribution: {pos_pct:.1f}% Positive, {neg_pct:.1f}% Negative, {neu_pct:.1f}% Neutral")

def combine_and_split_datasets():
    dataframes = []
    print("Reading input files:")
    for file in INPUT_FILES:
        try:
            df = pd.read_csv(file)
            dataframes.append(df)
            print(f"  - Loaded '{file}' with {len(df):,} rows.")
        except FileNotFoundError:
            print(f"  - Warning: '{file}' not found. Skipping.")
    
    if not dataframes:
        print("\nError: No data files found. Aborting.")
        return

    combined_df = pd.concat(dataframes, ignore_index=True)
    print(f"\nCombined dataset has a total of {len(combined_df):,} rows.")

    shuffled_df = combined_df.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)
    print("Dataset shuffled successfully.")

    train_df, temp_df = train_test_split(
        shuffled_df, test_size=0.2, random_state=RANDOM_STATE
    )

    validation_df, test_df = train_test_split(
        temp_df, test_size=0.5, random_state=RANDOM_STATE
    )
    
    print("\nSaving final CSV files")
    train_df.to_csv(OUTPUT_FILES["train"], index=False)
    validation_df.to_csv(OUTPUT_FILES["validation"], index=False)
    test_df.to_csv(OUTPUT_FILES["test"], index=False)

    print("\nProcess Complete")
    print_distribution(train_df, "train.csv")
    print_distribution(validation_df, "validation.csv")
    print_distribution(test_df, "test.csv")

if __name__ == "__main__":
    combine_and_split_datasets()

Reading input files:
  - Loaded 'cassa.csv' with 6,400 rows.
  - Loaded 'goemotions.csv' with 11,638 rows.
  - Loaded 'guiacat.csv' with 5,750 rows.

Combined dataset has a total of 23,788 rows.
Dataset shuffled successfully.

Saving final CSV files

Process Complete
'train.csv' (19,030 rows):
  Distribution: 40.5% Positive, 30.1% Negative, 29.4% Neutral
'validation.csv' (2,379 rows):
  Distribution: 38.1% Positive, 30.4% Negative, 31.4% Neutral
'test.csv' (2,379 rows):
  Distribution: 42.1% Positive, 28.2% Negative, 29.8% Neutral
